In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

## download the word vectors

In [ ]:
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
!pip3 install wikipedia-api


  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13477 sha256=6c218e0992737eba71f61273a3f06bd34a952412cd7125bc43aaaa10fc58b3e9
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


get data 

In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

In [ ]:
pages=[wiki_wiki.page('Cristiano Ronaldo'),
wiki_wiki.page('Lionel Messi'),
wiki_wiki.page('Mohamed Salah'),
wiki_wiki.page('Karim Benzema'),
wiki_wiki.page('Neymar'),
wiki_wiki.page('Samsung'),
wiki_wiki.page('Apple Inc.'),
wiki_wiki.page('OnePlus'),
wiki_wiki.page('Nokia'),
wiki_wiki.page('Xiaomi'),
wiki_wiki.page('Facebook'),
wiki_wiki.page('Instagram'),
wiki_wiki.page('WhatsApp'),
wiki_wiki.page('Snapchat'),
wiki_wiki.page('TikTok'),
wiki_wiki.page('The Weeknd'),
wiki_wiki.page('Sam Smith'),
wiki_wiki.page('Shawn Mendes'),
wiki_wiki.page('Justin Bieber'),
wiki_wiki.page('Zayn Malik'),
wiki_wiki.page('Paris'),
wiki_wiki.page('London'),
wiki_wiki.page('Kabul'),
wiki_wiki.page('Madrid'),
wiki_wiki.page('Ankara')]

In [ ]:
data={}
for i in pages:
  data[i.title]=i.text

In [ ]:
def preprocessing(data):
    import re
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stopwords=set(stopwords.words('english'))
    for i in data:
      data[i]=data[i].lower()
      data[i]=re.sub('\n', ' ', data[i])
      data[i]=re.sub('[^a-zA-Z\s]*', '', data[i])
      data[i]=data[i].split()
      data[i]=[word for word in data[i] if word not in stopwords]     
      
    return data  


In [ ]:
def Training(data):
    import numpy as np
    for i in data:
      lst=[]
      for j in data[i]:
        if(j in word2vec_model.vocab):
           lst.append(word2vec_model[j])
      data[i]=np.array(lst)  
    return data

In [ ]:
def avg(data):
    for i in data:
      data[i]=np.mean(data[i], axis = 0)
    return data  

In [ ]:
def main(data):
  data=preprocessing(data)
  data=Training(data)
  #make each doc as one vec
  data=avg(data)
  return data

In [ ]:
data=main(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.DataFrame.from_dict(data, orient="index")
df.to_csv("data.csv")


In [ ]:
dff = pd.read_csv("data.csv", index_col=0)
d = dff.to_dict("split")
d = dict(zip(d["index"], d["data"]))
for i in d:
  d[i]=np.array(d[i])

In [ ]:
mysentence={1:"capital of spain",2:"the best football player in world",3:"a British singer"}

In [ ]:
mysentence=main(mysentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from numpy import dot
from numpy.linalg import norm
for i in mysentence:
  result={}
  for j in data:
    #compute similarity by cosine similarity
    result[j] = dot(mysentence[i], data[j])/(norm(mysentence[i])*norm(data[j]))
  sort_orders = sorted(result.items(), key=lambda x: x[1], reverse=True)
  print(sort_orders)

[('Madrid', 0.59354836), ('Paris', 0.5418557), ('London', 0.53775483), ('Kabul', 0.47338113), ('Samsung', 0.4477142), ('Ankara', 0.43869993), ('Cristiano Ronaldo', 0.43330616), ('Lionel Messi', 0.42871916), ('Neymar', 0.4246755), ('Karim Benzema', 0.42206714), ('Nokia', 0.41587657), ('Mohamed Salah', 0.4121612), ('Xiaomi', 0.39238426), ('OnePlus', 0.37895733), ('Facebook', 0.35907766), ('Apple Inc.', 0.35819224), ('WhatsApp', 0.3455623), ('Sam Smith', 0.34451327), ('Zayn Malik', 0.34421867), ('Justin Bieber', 0.34166652), ('TikTok', 0.34031108), ('Shawn Mendes', 0.33523044), ('The Weeknd', 0.2965294), ('Instagram', 0.29374656), ('Snapchat', 0.2537795)]
[('Cristiano Ronaldo', 0.66101503), ('Lionel Messi', 0.6571004), ('Mohamed Salah', 0.6504365), ('Neymar', 0.6454509), ('Karim Benzema', 0.627401), ('Shawn Mendes', 0.46356764), ('Justin Bieber', 0.46152693), ('Zayn Malik', 0.4582121), ('The Weeknd', 0.4488836), ('Madrid', 0.44015992), ('Sam Smith', 0.43829978), ('London', 0.42733732), ('